
<H1 align = center >Convolution Neural Network을 감성분석에 적용하기</H1> 
<H3 align = center style="color:gray;" >Dr. Seong-K. Park@ai-khwarizmi.com</H3>


### 요약 정리

인터넷과 소셜 미디어 시대에 사람들의 견해와 리뷰, 추천은 정치나 비지니스에 상당히 중요한 자원이 되고있습니다. 현대 기술에 감사하게도 이제 우리는 이런 데이터를 대단히 효율적으로 수집하고 분석할 수 있는 기술이 생겼습니다. 이번 장에서는 감성분석이라고 불리는 자연어 처리(Natural Language Processing, NLP)의 상세 영역까지 들어가서 문서를 분류하기 위한 자연어 처리(NLP) 를 위한 컨볼류션알고리즘의 사용법을 설명합니다.

최근에 많은 애플리케이션에서 사용하는 데이터가 있는가 있는데, 바로 텍스트입니다. 스팸 메일 분류를 예로 들면, 이메일의 내용에 이 분류 작업에 필요한 중요한 정보가 들어 있습니다. 또는 이민 정책에 관한 정치인의 의견을 분석해야 할 때 각자의 언행이나 트윗이 중요한 정보를 제공합니다. 고객 서비스에서는 메시지가 불만사항인지 문의사항인지를 구분해야 할 때가 많습니다. 메시지의 제목이나 내용으로 고객의 의도를 자동으로 파악해서 적절한 부서로 전달하거나, 완전히 자동으로 응답할 수 도 있습니다.

<p style="color:blue; font-weight:bold;">
텍스트 데이터는 주로 글자가 연결된 문자열로 표현됩니다. 태생적으로 텍스트 데이터의 길이는 서로 같은 경우가 거의 없습니다. 이런 특성은 이제까지 본 수치형 데이터와 매우 다르므로 머신러닝 알고리즘에 적용하기 전에 신중하게 전처리를 해야 합니다. 그러나 모든 새로운 지식이 그렇듯이 이해하고 나시면 당연하게 생각하게 되실겁니다. 그리고 더 좋은 새로운 방법을 생각하게 되실 겁니다. 이유는 여러분들의 배경지식이 서로 다르기 때문입니다. 
</p>



여러분들의 이해를 돕기 위해 우선은 Keras에서 제공하는 전처리 과정이 완료된 인터넷 영화 자료(IMDb)와 CNN을 적용하여 소비자의 감성을 예측해 봅니다. 그러나 train 그리고 test 자료를 보면 의미를 알 수 없는 숫자로만 표현 되어 있습니다. 



문장들이 어떤 원리을 적용하여 숫자로 변환되었는지 궁금 할 것입니다. 혹시 저만 궁금한 건가요? 


Appendix에서 자세히 설명해 놓았으니 걱정하지 마시고 편안하게 읽으시면 됩니다.
혹시 나는 수식을 싫어한다고 생각하시는 독자 분들은 <font size = "2" style="font-weight:bold; color:black;"> Appendix : 단어들을 피처 벡터로 변환 </font>까지만 읽으셔도 어느 정도는 이해 하실 수 있을 겁니다. 그 이상은 읽지 않으셔도 됩니다. 



## Introduction

종종 오피니언 마이닝(opinion mining)이라고도 불리는 감성분석은 더 넓은 분야인 NLP에서도 인기 있는 분야이다. 감성분석은 문서의 양극성을 분류한다. 감성분석에서 주로 많이 하는 작업은 특성 토픽에 대해 작성자들의 의견이나 감성을 기반으로 분류하는 것이다.

이번 장에서는 인터넷 영화 자료(Internet Movie Database, IMDb)의 영화 리뷰 데이터로 작업을 할 것이다. 이 데이터는 매스 등에(Mass et al.)의해 수집되었다. 영화 리뷰 데이터는 5만 개로 양이나 음으로 레이블된 양극의 영화 리뷰로 구성되어 있다(여기서, 별 6개이상은 양(+) 그리고 아니면 음(-)). 다음 절에서는 이러한 영화 리뷰의 부분집합에서 의미 있는 축출하여 특정 리뷰어가 영화에 대해 좋다고 하는지 싫다고 하는지를 예측할 수 있는 머신러닝 모델을 만드는 방법에 대해 학습한다. 


#### CNN을 이용한 영화 리뷰 감성 분석 프로그램 작성

#### Header file and parameter를 입력합니다.

```python

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 2 * 32
embedding_dims = 50
filters = 250

kernel_size = 3
hidden_dims = 250
epochs = 2

```

### 영화 리뷰에 대한 BOW 데이터 로딩 
input 데이터의 사이즈가 서로 다르다는 것에 주의 하셔야 합니다.

예를 들면 
sequences = [
[1, 2, 3, 4],
[1, 2, 3],
[1]
]

```python

print('Loading data...')

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences'); print(len(x_test), 'test sequences')

print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))

결과 :

Loading data...
25000 train sequences
25000 test sequences

218
189
141

```

### 영화 리뷰에 대한 BOW 데이터 크기 맞추기 :
input 데이터의 사이즈를 동일하게 만듭니다.

예를 들면 

Before<br>
sequences = [ 
[1, 2, 3, 4], 
[1, 2, 3], 
[1] ]

After<br>
sequences = [
[1 2 3 4],
[0 1 2 3],
[0 0 0 1]]

```python

print('Pad sequences (samples x time)')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = sequence.pad_sequences( x_test, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print(len(x_train[0]))
print(len(x_train[1]))
print(len(x_train[2]))


결과 :
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)

400
400
400
```

#### 영화 리뷰을 위한 CNN model 만들기

```python

print('Build model...')
model = Sequential()


model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))

model.add(Conv1D(filters, kernel_size, padding='same', activation='relu'))
                 
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())
# Final evaluation of the model


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

결과:

Build model...
Epoch 1/2
25000/25000 [==============================] - 132s - loss: 0.4463 - acc: 0.7801   
Epoch 2/2
25000/25000 [==============================] - 131s - loss: 0.2702 - acc: 0.8888   

```

#### Train 된 영화 리뷰 자료를 test 셋에 적용하기

```python

scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

y_hat = model.predict_classes(x_test)

test_false = [im for im in zip(x_test,y_hat,y_test) if im[1] != im[2]]
test_true = [im for im in zip(x_test,y_hat,y_test) if im[1] == im[2]]
print('test_false: ',len(test_wrong))
print('test_true: ',len(test_true))


결과:
25000/25000 [==============================] - 14s   

Accuracy: 0.89%
test_false: 2727
test_true: 22273

```

# Appendix

## Training data와 Testing data를 만들기

### 진행 순서
<br>
영화 리뷰 데이터의 압축된 아카이브(84.1MB) 다운로드 장소:  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
<br>

데이터를 성공적으로 추출했다면 압축이 풀린 다운로드 아카이브에서 하나의 CSV 파일로 개별 텍스트 문서들을 모을 것입니다. 

<font size = "3" style="font-weight:bold; color:gray;"> 개별 텍스트를 하나의 CSV로 합치는 프로그램 (Make_imdb_csv.py) </font>

```python
import pandas as pd
import os

import numpy as np


Imdb_file = './Imdb_data.csv'

if os.path.exists(Imdb_file):
    
    df = pd.read_csv(Imdb_file)
    
    print(df.head(3))

else:
    labels = {'pos':1, 'neg':0}

    df = pd.DataFrame()

    for s in {'test', 'train'}:
        for l in ('pos', 'neg'):
            path = './aclImdb/%s/%s' % (s, l)
            for file in os.listdir(path):
                with open(os.path.join(path, file), 'r') as infile:
                    txt = infile.read()

                df = df.append([[txt, labels[l]]], ignore_index=True)

    df.columns = ['review','sentiment']

    np.random.seed(0)

    df = df.reindex(np.random.permutation(df.index))
    df.to_csv(Imdb_file, index=False)
```

이 코드를 실행하면 내장 for 루프를 사용하면 aclImbd 메인 폴더의 하위 디렉터리인 train과 test를 반복해서 개별 텍스트 파일을 하위 디렉터리 pos와 neg로부터 읽어들일 것입니다. 이것은 결국 DataFrame df에 정수 클래스 레이블과 함께 덧붙여집니다.

이 데이터의 클래스 레이블들은 정렬되어 있기 때문에 np.random 하위 모듈의 permutation 함수를 사용해서 DataFrame을 셔플링할 것입니다. 이것은 다음 코드에서 로컬 드라이브에서 데이터을 직접 스트리밍할 때 데이터를 훈련 데이터와 테스트 데이터로 나누는 데 유용하게 사용됩니다. 덧붙이고 셔플링한 영화 데이터를 나중의 편의를 위해 CSV 파일로 저장하도록 합니다.

## 텍스트 데이터를 Bag-of-words(BOW)로 표현하기

테스트나 워드와 같은 범주형 데이터를 머신러닝 알고리즘에 넣기 전에 수치형으로 변환해야 합니다. 이번 절에서는 테스트를 수치형 피처벡터롤 표현할 수 있게 해주는 BOW 모델의 개념을 소개하려고 합니다. BOW 모델의 개념은 간단해서 다음과 같이 요약할 수 있습니다. 

<ol>
<li>고유 토큰(token) 단어집(vocabulary)을 만든다.</li>
<li>특정 문서에서 각각의 단어가 얼마나 자주 사용되었는지 횟수를 포함하는 각 문서에 대해 피처 벡터를 만든다.</li>
</ol>

각 문서의 고유 단어들은 단지 BOW 단어집 내 모든 단어들에 대한 하나의 작은 부분집합을 나타낼 뿐이기 때문에 피처 벡터는 대부분 0이 됩니다. 

#### 단어들을 피처 벡터로 변환

각각의 문서 내에서 단어가 사용된 숫자에 기반하여 BOW 모델을 만들기 위해 사이킷런에서 구축했던 CountVectorizer 클래스를 사용할 수 있습니다. 다음 코드에서 보게 될 CountVectorizer 클래스는 텍스트 데이터를 배열로 취해서(문장 또는 문서) BOW 모델을 생성합니다.


```python
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array(['The sun is shining', 'The weather is sweet', 'The sun is shining and the weather is sweet'])

bag = count.fit_transform(docs)

labels = count.vocabulary_

print(labels);
{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}

print(bag.toarray())
[[0 1 1 1 0 1 0] [0 1 0 0 1 1 1] [1 2 1 1 1 2 1]]

```
<br>

출력은 각 문서에서 나타난 단어의 횟수가 담긴 벡터입니다. 이를 위해 사전에 있는 각 단어가 문서마다 얼마나 자주 나타나는지 세어야 합니다. 즉, 이 수치 표현은 전제 데이터셋에서 고유한 각 단어를 특성으로 가집니다. 원본 문자열에 있는 단어의 순서는 BOW 특성 표현에서는 완전히 무시됩니다.

여기서 볼 수 있는 피처 벡터에서 각 인덱스의 위치는 CountVectorizer 단어집의 사전 아이템에 저장되어 있는 정수에 대응하는 것이다. 예를 들어, 인데스 위치의 첫 번째 피처 0은 마지막 문서에서만 발생한 단어 and의 수처럼 생겼고, 인덱스 위치 1에 있는 단어 is는 (문서 벡터에서 두 번째 피처) 전체 세 문장에서 발생한다. 피처 벡터내의 이 값들을 raw-term-frequencies이라고 한다(tf(t,d)-문서 d에서 단어가 나타나는 횟수 t번). 

#### 용어 빈도수 - 문서 빈도수의 역수를 이용한 단어의 관련성 평가

테스트 데이터를 분석할 때 종종 여러 개의 분서에서 발생하는 단어들을 만나게 된다. 이들은 일반적으로 유용한 정보나 차별적인 정보를 포함하지 않고 자주 발행하는 단어들이다. 이번 절에서는 term frequency-inverse document frequency(if-idf)라는 유용한 기술에 관해 설명하려 한다. 이것은 피처 벡터에서 빈번하게 발생하는 단어들의 가중치를 낮추는 데 사용된다. tf-idf 용어 빈도수와 문서 빈도수의 역수의 곱으로 정의할 수 있다.
<br>
<center>
tf-idf(t,d) = $\text{tf(t,d)} \times \left( \text{idf(t,d)+1}\right)$
</center>
<br>

여기서 tf(t,d)는 용어 빈도수이고 문서 빈도수의 역수 idf(t,d)는 다음과 같이 계산할 수 있다.
<br>
<center>
idf(t,d) = $log\frac{1+n_d}{1+\text{df}(d,t)}$,
</center>
<br>

여기서 $n_d$는 문서의 전체 개수이며, df(d,t)는 용어 t를 포함하는 문서의 d의 개수이다. 이 상수 1을 분모에 더하는 것은 옵션은 학습 샘플 모두에 대해 분모가 0이 되지 않도록 만들어 역할을 한다는 것에 유념하다. 로그는 문서의 빈도가 낮은 경우 가중치가 너무 높아지지 않도록 해준다.

사이킷런은 TfidfTransformer라는 또 하나의 함수를 가지고 있다. 이것은 CountVectorizer에서 raw 항의 빈도수를 입력값으로 하며 이것은 tf-idf로 변형할 수 있다.


```python
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


count = CountVectorizer()

docs = np.array(['The sun is shining', 'The weather is sweet', 'The sun is shining and the weather is sweet'])

bag = count.fit_transform(docs)

labels = count.vocabulary_

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

np.set_printoptions(precision=2)


print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.  0.43 0.56 0.56 0.   0.43 0.  ] 
 [ 0.  0.43 0.   0.   0.56 0.43 0.56] 
 [ 0.4 0.48 0.31 0.31 0.31 0.48 0.31]]
```

#### 텍스트 데이터 정제

BOW 모델을 만들기 전에 중요한 단계는, 예측의 정확도를 높이기 위해 불필요한 잡음(노이즈)를 제거 하겠습니다. 문자나 문장에서 노이즈를 없앤다는 것은 다시 말하면 원하지 않는 문자(특수문자, 문서에 섞여 있는 html, etc)를 모두 가려내어 텍스트 데이터를 편집하는 것 입니다. 
<br><br>
여기서는 간단한 예제로 언급만 하겠습니다.

```python
import numpy as np
import re
from nltk.corpus import stopwords

# import nltk
# nltk.download("stopwords")

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]

    return tokenized

print(tokenizer('a runner likes running and runs a lot @.@'))

['runner', 'likes', 'running', 'runs', 'lot']
```

# 결론